In [1]:
import pandas as pd
import numpy as np
import os
import shutil
from tqdm import tqdm
from glob import glob
import librosa

In [2]:
# glob로 test data의 path를 불러옴. 순서대로 로드되지 않을 경우를 주의
# test_ 데이터 프레임을 만들어 나중에 sample_submission과 id를 기준으로 merge시킬 준비

def get_id(data):
    return int(os.path.split(data)[-1].split(".")[0])

test_ = pd.DataFrame(index = range(0, 6100), columns = ["path", "id"])
test_["path"] = glob("./data/test/*.wav")
test_["id"] = test_["path"].apply(lambda x : get_id(x))

test_.head()

,path,id
0,./data/test/4737.wav,4737
1,./data/test/793.wav,793
2,./data/test/1961.wav,1961
3,./data/test/5574.wav,5574
4,./data/test/1618.wav,1618


In [3]:
train_x = np.load("./npy_data/train_x.npy")
train_y = np.load("./npy_data/train_y.npy")
test_x = np.load("./npy_data/test_x.npy")

train_x = train_x.reshape((25520, 64 * 501))
test_x = test_x.reshape((6100, 64 * 501))

print(train_x.shape, train_x.dtype)
print(train_y.shape, train_y.dtype)
print(test_x.shape, test_x.dtype)

(25520, 32064) float32
(25520,) int64
(6100, 32064) float32


In [4]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(train_x)


MinMaxScaler()

In [5]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(train_x, train_y, test_size=0.2)

print(train_x.shape)
print(test_x.shape)
print(train_y.shape)
print(test_y.shape)


(20416, 32064)
(5104, 32064)
(20416,)
(5104,)


# ML MODEL

In [14]:
# LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

LR = LogisticRegression()
LR.fit(train_x, train_y)
pred = LR.predict(test_x)
acc = accuracy_score(pred, test_y)
print(acc)


/home/hoyun/.conda/envs/tf/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
0.4715909090909091


In [7]:
# DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

DT = DecisionTreeClassifier()
DT.fit(train_x, train_y)
pred = DT.predict(test_x)
acc = accuracy_score(pred, test_y)
print(acc)


0.3906739811912226


In [7]:
# RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

RF = RandomForestClassifier(n_estimators=100, oob_score=True, random_state=0)
RF.fit(train_x, train_y)
pred = RF.predict(test_x)
acc = accuracy_score(pred, test_y)
print(acc)

0.538205329153605


In [8]:
# AdaBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

AB = RandomForestClassifier(n_estimators=50)
AB.fit(train_x, train_y)
pred = AB.predict(test_x)
acc = accuracy_score(pred, test_y)
print(acc)

0.5211598746081505
